# Week 9 Class 1 - NonLinear Programming

We we begin our pyomo solving of nonlinear programs with the nonlinear solver called `ipopt`. You can think of it like GRG Nonlinear in Excel.  You will call it instead of the linear solver `glpk` that we have been using.

### Example 8.1 - Crowley Foods 

In this example, we have a straightforward formula for the objective function and a single constraint. We will just build this model in pyomo (won't read in data from Excel).

In [ ]:
import pyomo.environ as pe
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
#Define the model parameters - makes it easy to understand the objective function formula
fixed_cost = 35
unit_cost = 30
carrying = .1
carrying_cost = 3
annual_demand = 5000

In [ ]:
df = pd.DataFrame()
df['lot_size'] = np.arange(250, 500, 5)
df['total_cost'] = fixed_cost*annual_demand/df['lot_size'] + carrying_cost * df['lot_size']/2

In [ ]:
plt.figure(figsize = (8,5))
sns.lineplot(x = 'lot_size', y = 'total_cost', data = df)
plt.title('Annual Cost (y) versus Lot Size (x)')
plt.ylabel('total cost')
plt.xlabel('lot size')
plt.grid()
plt.show()

In [ ]:
model = pe.ConcreteModel()

model.lot_size = pe.Var(domain = pe.NonNegativeReals)

#Define the Objective Function
model.obj = pe.Objective(expr = fixed_cost*annual_demand/model.lot_size +
                         carrying_cost * model.lot_size/2,
                         sense = pe.minimize)
model.obj.pprint()

In [ ]:
model.const = pe.Constraint(expr = model.lot_size >= 1)
model.const.pprint()

In [ ]:
opt = pe.SolverFactory('ipopt')
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

In [ ]:
obj_val = model.obj.expr()
print(f'optimal objective value = {obj_val:.2f}')

In [ ]:
opt_lot_size = model.lot_size.value
print(f'optimal lotsize = {opt_lot_size:.2f}')